In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import random

In [2]:
np.random.seed(10)

### Generate Shapes

In [24]:
def create_rectangle(top_left, bottom_right, size):
    R = np.zeros((size, size), dtype = np.uint8)
    for i in range(top_left[0], bottom_right[0]+1):
        for j in range(top_left[1], bottom_right[1]+1):
            R[[i],[j]] = 1
            
    return R

def create_circle(top_left, bottom_right, size):
    C = np.zeros((size, size), dtype = np.uint8)
    for i in range(top_left[0], bottom_right[0]+1):
        for j in range(top_left[1], bottom_right[1]+1):
            x = int(round((top_left[0]+bottom_right[0])/2))
            y = int(round((top_left[1]+bottom_right[1])/2))
            r = x - top_left[0]
            if np.square(i - x)+np.square(j - y) <= np.square(r):
                C[[i],[j]] = 1
                
    return C

def create_random_shape(top_left, bottom_right, size):
    i = random.randint(1,2)
    if i==1:
        return create_circle(top_left, bottom_right, size)
    elif i==2:
        return create_rectangle(top_left, bottom_right, size)
    elif i==3:
        return create_triangle(top_left, bottom_right, size)

### Color the shapes randomly

In [25]:
def create_random_color(Gray, size):
    r=np.random.randint(1,255)
    g=np.random.randint(1,255)
    b=np.random.randint(1,255)
    C_shape=np.concatenate((r*Gray,g*Gray,b*Gray))
    C_shape=C_shape.reshape((3,size,size))
    
    return C_shape

In [26]:
n = 20000 #number of arrays we want to create

split = 18000 #how much we want to use for training
epos = 40

size = 100

shapes = np.zeros((n, 3, size, size))
shapes_attributes = np.zeros((n, 4))

#create random dataset
for i in range(n):
    x_top = np.random.randint(10,(size-30))
    y_left = np.random.randint(10,(size-30))
    top_left = (x_top, y_left)

    #x_bottom = np.random.randint((x_top+5), (size-10))    #not good for circles
    #y_right = np.random.randint((y_left+5), (size-10))
    #bottom_right = (x_bottom, y_right)
    
    x_bottom = np.random.randint((x_top+5), (size-10))     #both axis same lenght
    y_right = np.random.randint((y_left+5), (size-10))
    x = x_bottom - x_top
    y = y_right - y_left
    lenght = min(x,y)
    x_bottom = x_top + lenght
    y_right = y_left + lenght
    bottom_right = (x_bottom, y_right)
    
    
    attributes = np.array([x_top, y_left, x_bottom, y_right])
    shapes_array = create_random_color(create_random_shape(top_left, bottom_right, size), size)
    
    shapes[i] = shapes_array
    shapes_attributes[i] = attributes

In [27]:
train_shapes, test_shapes = shapes[:split], shapes[split:]
train_attributes, test_attributes = shapes_attributes[:split], shapes_attributes[split:]

In [1]:
print(train_shapes.shape)
print(train_attributes.shape)
plt.imshow(shapes_array[1]) #just r-channel from rgb
train_shapes = train_shapes.reshape(split, size, size, 3)
test_shapes = test_shapes.reshape(n-split, size, size, 3)
print(train_shapes.shape)
print(test_shapes.shape)

NameError: name 'train_shapes' is not defined

In [9]:
model = keras.Sequential([
    keras.layers.Convolution2D(filters = 6, kernel_size = 25,data_format="channels_last", input_shape = (size, size, 3)),
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Convolution2D(10, (3,3)),
    keras.layers.Convolution2D(16, (3,3)),
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Convolution2D(8, (3,3)),
    keras.layers.Convolution2D(8, (3,3)),
    keras.layers.Flatten(),
    keras.layers.Dense(4)
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 76, 76, 6)         11256     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 38, 38, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 36, 10)        550       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 16)        1456      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 17, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 8)         1160      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 8)         5

In [11]:
model.compile(loss= "mean_squared_error" , optimizer="adam")

In [14]:
model.fit(train_shapes, train_attributes, epochs=epos)

Epoch 1/40
250/250 [==============================] - 230s 921ms/step - loss: 439.2700
Epoch 2/40
250/250 [==============================] - 230s 921ms/step - loss: 241.2828
Epoch 3/40
250/250 [==============================] - 230s 918ms/step - loss: 100.9077
Epoch 4/40
250/250 [==============================] - 230s 919ms/step - loss: 57.4745
Epoch 5/40
250/250 [==============================] - 230s 921ms/step - loss: 49.1298
Epoch 6/40
250/250 [==============================] - 231s 924ms/step - loss: 45.5324
Epoch 7/40
250/250 [==============================] - 230s 921ms/step - loss: 44.0055
Epoch 8/40
250/250 [==============================] - 230s 920ms/step - loss: 3883.1833
Epoch 9/40
250/250 [==============================] - 237s 948ms/step - loss: 1010.3616
Epoch 10/40
250/250 [==============================] - 234s 936ms/step - loss: 313.3460
Epoch 11/40
250/250 [==============================] - 234s 937ms/step - loss: 239.8663
Epoch 12/40
250/250 [======================

In [15]:
model.evaluate(test_shapes, test_attributes)

63/63 [==============================] - 7s 105ms/step - loss: 48.1240


48.124027252197266

In [18]:
predictions = model.predict(test_shapes)

for i in range(10):
    print(max(abs(predictions[i] - test_attributes[i])))
    
for i in range(4):
    print(predictions[i])
    print(test_attributes[i])

4.269142150878906
7.034528732299805
14.224342346191406
7.548139572143555
7.2963714599609375
4.23907470703125
3.0891075134277344
4.297458648681641
9.152961730957031
5.700721740722656
[13.029562 33.535656 33.269142 56.203495]
[12. 37. 29. 54.]
[31.034529 33.6922   54.908348 57.47564 ]
[24. 31. 50. 57.]
[47.260082 38.22215  72.77566  64.8303  ]
[49. 38. 87. 76.]
[27.54814  56.900566 38.483597 69.032394]
[20. 53. 31. 64.]
